<a href="https://colab.research.google.com/github/NagypalMarton/Speaking-with-LLM-in-mother-tongue/blob/main/voice_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Feladat szövege**


---


A cél egy olyan hangalapú chatbot létrehozása, amellyel időpontot lehet foglalni egy szolgáltatóhoz. A párbeszédben a legfontosabb információkat kell megkérdezni a megerősítéshez. A felhasználó azonosítása (név, e-mail, telefonszám vagy más módszer), időpont, választott kezelés vagy szolgáltatás, egyéb kért extra információk (pl. érzelmi állapot). Az eredményeknek json formátumban kell rendelkezésre állniuk. A rendszernek angol és/vagy magyar nyelven kell működnie.

https://huggingface.co/microsoft/Phi-4-mini-instruct

https://huggingface.co/microsoft/speecht5_tts

https://github.com/openai/whisper

In [1]:
!pip install --quiet transformers accelerate torch gradio faiss-cpu sentence-transformers openai-whisper

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, SpeechT5Processor, SpeechT5ForTextToSpeech
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr
import whisper
from datasets import load_dataset
import soundfile as sf

In [3]:
#STT/ASR (whisper) modell kiválasztása
#whisper_model = whisper.load_model("base")  # vagy "small", "medium", "large"
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:09<00:00, 15.9MiB/s]


In [8]:
#TTS Model inicializálás
tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# Load the dataset, allowing the library to use its default cache directory
# Removing cache_dir=None resolves the LocalFileSystem issue
# Explicitly set cache_dir to a location within /content for Google Colab
from datasets import load_dataset, DownloadMode
embeddings_dataset = load_dataset(
    "Matthijs/cmu-arctic-xvectors",
    split="validation",
    cache_dir="/content/huggingface_cache",
    download_mode=DownloadMode.FORCE_REDOWNLOAD # Added to force redownload
)
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
#TTS
def synthesize_speech_speecht5(text, output_path="tts_output.wav", speaker_embedding=speaker_embedding, sample_rate=16000):
    """
    Szöveg -> hangfájl (wav) (SpeechT5)
    """
    inputs = tts_processor(text=text, return_tensors="pt")
    speech = tts_model.generate_speech(inputs["input_ids"], speaker_embedding)
    sf.write(output_path, speech.cpu().numpy(), sample_rate)
    return output_path

In [ ]:
# 3. GPU detektálása
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
#Speech-To-Text
def transcribe_audio_whisper(audio_path, lang="hu"):
    """
    Felvett hangfájl (wav) -> szöveg (Whisper).
    """
    result = whisper_model.transcribe(audio_path, language=lang)
    return result["text"]

In [ ]:
# 4. Dokumentumok
documents = [
    "Google Colab egy egyszerű felhős Jupyter notebook környezet, ahol GPU is elérhető.",
    "A Gradio egy Python könyvtár, amely gyorsan lehetővé teszi AI modellek webes UI-jának létrehozását.",
    "A RAG technológia ötvözi a kereső (retriever) és generatív (generator) modelleket, hogy pontosabb válaszokat adjon.",
    "A Hugging Face modellek könnyen letölthetők és használhatók különböző NLP feladatokra.",
    "A phi-4 egy erős, open-source nyelvi modell a Microsofttól, amely képes természetes nyelvi kérdésekre válaszolni."
]

In [ ]:
# 5. Embedding modell és indexelés
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

In [ ]:
# 6. LLM betöltése (Phi-4-mini-instruct)
model_id = "microsoft/phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

In [ ]:
#Zero Shot Intent Detection
def detect_intent_llm(user_utterance, lang="hu"):
    """
    Zero-shot intent detection LLM-mel.
    user_utterance: a felhasználó mondata
    lang: 'hu' vagy 'en'
    Visszaadott intent: szöveg (pl. 'foglalás', 'info', stb.)
    """
    if lang == "hu":
        prompt = (
            "Az alábbi mondat alapján válaszd ki a szándékot az alábbiak közül (csak egy szót adj vissza, szóköz nélkül): "
            "foglalás, lemondás, módosítás, információ, köszönés, egyéb.\n"
            f"Felhasználó: {user_utterance}\nSzándék:"
        )
    else:
        prompt = (
            "Based on the following sentence, choose the intent from: booking, cancel, reschedule, info, greeting, other. "
            "Return only one word, no spaces.\n"
            f"User: {user_utterance}\nIntent:"
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=3)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Szándék kiszűrése a válaszból
    intent = answer.split(":")[-1].strip().split()[0]
    return intent

In [ ]:
def rag_phi4mini_chatbot(user_input, input_mode="text", lang="hu"):
    """
    Chatbot, ami támogatja a hang- és szövegalapú chatet is.
    user_input: szöveg (input_mode="text") vagy hangfájl útvonal (input_mode="audio")
    input_mode: 'text' vagy 'audio'
    lang: 'hu' vagy 'en'
    return: (válasz_szöveg, intent, válasz_hangfájl_útvonal)
    """
    # 1. Ha hang, futtasd le a STT-t
    if input_mode == "audio":
        user_utterance = transcribe_audio_whisper(user_input, lang=lang)
    else:
        user_utterance = user_input

    # 2. Intent detektálás
    intent = detect_intent_llm(user_utterance, lang=lang)

    # 3. Embed és keresés (RAG)
    q_embed = embedder.encode([user_utterance])
    D, I = index.search(np.array(q_embed), k=1)
    context = documents[I[0][0]]

    # 4. Prompt építés
    if D[0][0] > 1.0:
        prompt = f"Kérdés: {user_utterance}\nSzándék: {intent}\nVálasz:"
    else:
        prompt = f"""
Az alábbi kontextus alapján válaszolj a kérdésre magyarul:

KONTEKSTUS:
{context}

KÉRDÉS:
{user_utterance}

SZÁNDÉK:
{intent}

VÁLASZ:
"""
    # 5. LLM generálás
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "VÁLASZ:" in answer:
        answer = answer.split("VÁLASZ:")[-1].strip()
    elif "Válasz:" in answer:
        answer = answer.split("Válasz:")[-1].strip()

    # 6. TTS – szövegből hangfájlt generálunk
    tts_output_path = synthesize_speech_speecht5(answer)  # Ez visszaadja a .wav file útvonalát

    return answer, intent, tts_output_path

In [ ]:
# 8. Gradio UI
iface = gr.Interface(
    fn=rag_phi4mini_chatbot,
    inputs=gr.Textbox(lines=2, label="Kérdés magyarul"),
    outputs=gr.Textbox(label="Chatbot válasza"),
    title="RAG Chatbot - Microsoft Phi-4-mini-instruct + Gradio (Google Colab, GPU)",
    description="Kérdezz bátran! A chatbot a kontextus alapján válaszol magyarul."
)

iface.launch(share=True)